In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import levitate
import levitate.hardware
import ipywidgets
progressbar = ipywidgets.IntProgress(value=0,min=0,max=10,
    description='Calculating: ',bar_style='', orientation='horizontal')

In [ ]:
levitate.models.update_air_properties(temperature=25)
transducer = levitate.models.CircularRing(effective_radius=2.9e-3, freq=40e3)
# transducer = levitate.models.ReflectingTransducer(levitate.models.CircularRing, plane_distance=0.21, plane_normal=(0,0,1), effective_radius=2.9e-3, freq=40e3)
array = levitate.models.TransducerArray(transducer_model=transducer, grid=levitate.hardware.dragonfly_grid())

In [ ]:
filenames = 'two_bead_circle_prevuse/{}.dat'
complex_amplitudes = levitate.hardware.data_from_cpp(filenames.format('two_beads'), array.num_transducers)
first_bead_amplitudes = levitate.hardware.data_from_cpp(filenames.format('first_bead'), array.num_transducers)
second_bead_amplitudes = levitate.hardware.data_from_cpp(filenames.format('second_bead'), array.num_transducers)
first_bead_nulled_amplitudes = levitate.hardware.data_from_cpp(filenames.format('first_bead_null'), array.num_transducers)
second_bead_nulled_amplitudes = levitate.hardware.data_from_cpp(filenames.format('second_bead_null'), array.num_transducers)

In [ ]:
center_loc = np.array([0,0,0.05])
radius = 30e-3
# number_of_points = 360*3
resolution = 0.1e-3
resolution = 0.1
number_of_points = round(2*np.pi*radius/resolution)
number_of_points = 1
angular_resolution = 2*np.pi/number_of_points
angles = np.arange(number_of_points) * angular_resolution
# locations = radius * np.cos(angles)[:,np.newaxis] * np.stack([np.cos(angles), np.sin(angles), np.zeros(angles.shape)], axis=1) + center_loc
locations = []
complex_amplitudes = []

progressbar.max = number_of_points-1
display(progressbar)

optim = levitate.optimization.Optimizer(array)
optim.variable_amplitudes = True

for point in range(number_of_points):
    progressbar.value = point
    
    angle = point * angular_resolution
    locations.append(center_loc + radius * np.array([np.cos(angle),np.sin(angle), 0]))
    gorkov = levitate.optimization.gorkov_laplacian(array, location=locations[-1], weights=(1e-3, -1, -1, -1000))
    
    if point > 0:
        array.complex_amplitudes = complex_amplitudes[-1]
    else:
        array.phases = array.focus_phases(locations[-1]) + array.bottle_signature(locations[-1]) + 0.5*np.random.uniform(-np.pi, np.pi, array.num_transducers)
        array.amplitudes[:] = 0
    optim.objectives = [gorkov]
    optim()
    complex_amplitudes.append(optim.complex_amplitudes)
    
levitate.hardware.data_to_cpp(complex_amplitudes, 'output_phases.dat')

In [ ]:
center_loc = np.array([0,0,0.06])
radius = 30e-3
resolution = 0.1e-3
number_of_points = round(2*np.pi*radius/resolution)
number_of_points = 1
angular_resolution = 2*np.pi/number_of_points
first_bead_locations = []
second_bead_locations = []

first_bead_amplitudes = []
second_bead_amplitudes = []
first_bead_nulled_amplitudes = []
second_bead_nulled_amplitudes = []
combined_amplitudes = []
array.phases = array.focus_phases(center_loc) + array.twin_signature(center_loc)

progressbar.max = number_of_points-1
display(progressbar)

optim = levitate.optimization.Optimizer(array)
optim.variable_amplitudes = True

for point in range(number_of_points):
    progressbar.value = point
    
    first_bead_locations.append(center_loc + (radius*(np.cos(point * angular_resolution)), radius*np.sin(point * angular_resolution), 0))
    second_bead_locations.append(center_loc)
    gorkov_1 = levitate.optimization.gorkov_laplacian(array, location=first_bead_locations[-1], weights=(1e-3, -1, -1, -200))
    gorkov_2 = levitate.optimization.gorkov_laplacian(array, location=second_bead_locations[-1], weights=(1e-3, -1, -1, -200))
    null_1 = levitate.optimization.pressure_null(array, location=first_bead_locations[-1], weights=(1e-3, 1e-3, 1e-3, 1e-3))
    null_2 = levitate.optimization.pressure_null(array, location=second_bead_locations[-1], weights=(1e-3, 1e-3, 1e-3, 1e-3))
    
    if point == 0:
        array.phases = array.focus_phases(first_bead_locations[-1]) + array.bottle_signature(first_bead_locations[-1]) + 0.2 * np.random.uniform(-np.pi, np.pi, array.num_transducers)
        array.amplitudes[:] = np.random.uniform(0.5, 1, array.num_transducers)
    else:
        array.complex_amplitudes = first_bead_amplitudes[-1]
    optim.objectives = [gorkov_1]
    optim()
    first_bead_amplitudes.append(optim.complex_amplitudes)
    optim.objectives = [gorkov_1, null_2]
    optim()
    first_bead_nulled_amplitudes.append(optim.complex_amplitudes)
    
    if point == 0:
        array.phases = array.focus_phases(second_bead_locations[-1]) + array.bottle_signature(second_bead_locations[-1]) + 0.2 * np.random.uniform(-np.pi, np.pi, array.num_transducers)
        array.amplitudes[:] = np.random.uniform(0.5,1, array.num_transducers)
    else:
        array.complex_amplitudes = second_bead_amplitudes[-1]
    optim.objectives = [gorkov_2]
    optim()
    second_bead_amplitudes.append(optim.complex_amplitudes)
    optim.objectives = [gorkov_2, null_1]
    optim()
    second_bead_nulled_amplitudes.append(optim.complex_amplitudes)
    
    combined_amplitudes.append(0.5 * (first_bead_nulled_amplitudes[-1] + second_bead_nulled_amplitudes[-1]))
    
levitate.hardware.data_to_cpp(combined_amplitudes, 'two_beads.dat')
levitate.hardware.data_to_cpp(first_bead_amplitudes, 'first_bead.dat')
levitate.hardware.data_to_cpp(second_bead_amplitudes, 'second_bead.dat')
levitate.hardware.data_to_cpp(first_bead_nulled_amplitudes, 'first_bead_null.dat')
levitate.hardware.data_to_cpp(second_bead_nulled_amplitudes, 'second_bead_null.dat')

In [ ]:
loc_idx = 0
p_scale = lambda p: 20*np.log10(np.abs(p)) + 94
p_range = (140, 160)

# loc = locations[loc_idx]
# array.complex_amplitudes = complex_amplitudes[loc_idx]

loc = first_bead_locations[loc_idx]
# loc = np.array((0,0,0.06))
array.complex_amplitudes = first_bead_nulled_amplitudes[loc_idx]

resolution = array.wavelength/50
xz_mesh = np.meshgrid(np.arange(-0.08, 0.08, resolution), np.arange(0, 0.1, resolution), indexing='ij')
xz_mesh = np.meshgrid(np.arange(0.02, 0.04, resolution), np.arange(0.04, 0.08, resolution), indexing='ij')
pressures_xz = np.squeeze(array.calculate_pressure((xz_mesh[0], loc[1]*np.ones_like(xz_mesh[0]), xz_mesh[1])))

xy_mesh = np.meshgrid(np.arange(-0.08, 0.08, resolution), np.arange(-0.08, 0.08, resolution), indexing='ij')
xy_mesh = np.meshgrid(np.arange(0.02, 0.04, resolution), np.arange(-0.01,0.01, resolution), indexing='ij')
pressures_xy = np.squeeze(array.calculate_pressure((xy_mesh[0], xy_mesh[1], loc[2]*np.ones_like(xy_mesh[0]))))

fig = plt.figure()
plt.pcolormesh(xz_mesh[0], xz_mesh[1], p_scale(pressures_xz))
plt.xlabel('x')
plt.ylabel('z')
plt.clim(p_range)
plt.colorbar()
plt.plot(loc[0], loc[2], color='red', marker='o', markerfacecolor='red')
fig.show()

fig = plt.figure()
plt.pcolormesh(xy_mesh[0], xy_mesh[1], p_scale(pressures_xy))
plt.xlabel('x')
plt.ylabel('y')
plt.clim(p_range)
plt.colorbar()
plt.plot(loc[0], loc[1], color='red', marker='o', markerfacecolor='red')
fig.show()

In [ ]:
fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.signature(focus=loc)/np.pi, s=144, cmap='hsv')
plt.clim((-1,1))
plt.colorbar()
plt.title('Phase')
fig.show()

fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.amplitudes, s=144)
plt.colorbar()
plt.title('Amplitude')
fig.show()
plt.show()